# 식품영양성분_with_tags.csv => Pinecone에 적재 후 LLM 호출

In [5]:
import os
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI 

In [6]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [8]:
assert OPENAI_API_KEY and PINECONE_API_KEY, "API 키를 .env에 설정하세요."

# OpenAI / Pinecone 클라이언트
oai = OpenAI(api_key=OPENAI_API_KEY)
pc  = Pinecone(api_key=PINECONE_API_KEY)

INDEX_NAME = "food-test1"   # 인덱스 이름
NAMESPACE  = "foods-ns1"    # namespace (선택)

# 인덱스 생성(없으면)
try:
    # v5 SDK: .names() 지원
    names = pc.list_indexes().names()
except Exception:
    # 구버전 호환
    names = [x.name for x in pc.list_indexes()]

if INDEX_NAME not in names:
    pc.create_index(
        name=INDEX_NAME,
        dimension=1536,             # text-embedding-3-small 차원
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(INDEX_NAME)

# 전처리 완료 CSV 경로(파일명 꼭 확인!)
CSV_PATH = "식품영양성분_with_tags.csv"  # ← 네가 마지막에 만든 파일명으로
df = pd.read_csv(CSV_PATH, encoding="utf-8-sig")
print(f"🍱 총 {len(df)}개 로드 / 컬럼 예시: {list(df.columns)[:10]}")

🍱 총 548개 로드 / 컬럼 예시: ['food_id', 'food_name', 'ref_amount', 'energy_kcal', 'protein_g', 'fat_g', 'carb_g', 'sugar_g', 'sodium_mg', 'chol_mg']


In [9]:
def make_text(row):
    return f"""
    음식명: {row['food_name']} ({row['ref_amount']}g 기준)
    칼로리: {row['energy_kcal']}kcal
    단백질: {row['protein_g']}g, 지방: {row['fat_g']}g, 탄수화물: {row['carb_g']}g, 당: {row['sugar_g']}g
    나트륨: {row['sodium_mg']}mg, 콜레스테롤: {row['chol_mg']}mg
    포화지방: {row['sat_fat_g']}g, 트랜스지방: {row['trans_fat_g']}g
    중량: {row['food_weight']}g
    """.strip()
# ----------------------------
#  메타데이터 구성 함수 수정 (영양 정보 추가)
# ----------------------------
def make_metadata(row):
    return {
        "norm_tags": row.get("norm_tags", ""),
        "display_tags": row.get("display_tags", ""),
        "content": row.get("content", ""),
        "food_name": row.get("food_name", ""),
        "energy_kcal": row.get("energy_kcal", 0.0),
        "protein_g": row.get("protein_g", 0.0),
        "fat_g": row.get("fat_g", 0.0),
        "carb_g": row.get("carb_g", 0.0),
        "food_weight": row.get("food_weight", 0.0),
    }


In [10]:
# ----------------------------
#  Pinecone 배치 업로드
# ----------------------------
client = oai
batch_size = 100  # 🚀 한번에 100개씩 임베딩 + 업로드
for i in tqdm(range(0, len(df), batch_size), desc="🍽️ Pinecone 업로드 중"):
    batch = df.iloc[i:i + batch_size]

    # 1. 텍스트 리스트 생성
    batch_texts = [make_text(row) for _, row in batch.iterrows()]

    # 2. OpenAI에 배치 임베딩 요청 (한 번에 100개)
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=batch_texts
    )
    embeddings = [d.embedding for d in response.data]

    # 3. Pinecone 벡터 리스트 구성
    batch_vectors = []
    for (idx, row), emb in zip(batch.iterrows(), embeddings):
        meta = make_metadata(row)
        batch_vectors.append({
            "id": str(row["food_id"]),
            "values": emb,
            "metadata": meta
        })

    # 4. Pinecone 업서트 (namespace 지정)
    index.upsert(vectors=batch_vectors, namespace=NAMESPACE)

    print(f"  → {i + len(batch_vectors)}/{len(df)} 업로드 완료")

print(f"\n✅ Pinecone 업로드 완료! ({len(df)}개 항목, namespace='{NAMESPACE}')")

🍽️ Pinecone 업로드 중:  17%|█▋        | 1/6 [00:16<01:20, 16.13s/it]

  → 100/548 업로드 완료


🍽️ Pinecone 업로드 중:  33%|███▎      | 2/6 [00:20<00:36,  9.23s/it]

  → 200/548 업로드 완료


🍽️ Pinecone 업로드 중:  50%|█████     | 3/6 [00:25<00:21,  7.11s/it]

...

  → 400/548 업로드 완료


🍽️ Pinecone 업로드 중:  83%|████████▎ | 5/6 [00:35<00:06,  6.10s/it]

  → 500/548 업로드 완료


🍽️ Pinecone 업로드 중: 100%|██████████| 6/6 [00:38<00:00,  6.36s/it]

  → 548/548 업로드 완료

✅ Pinecone 업로드 완료! (548개 항목, namespace='foods-ns1')


In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [13]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7, 
    max_tokens=1024, 
    openai_api_key=OPENAI_API_KEY,
)

### 아래 두 셀은 실행 X -> 바로 수정된 프롬프트로 넘어가기

In [ ]:
import datetime
today = datetime.date.today().strftime("%Y-%m-%d")

prompt_template = f"""
당신은 최고 수준의 영양사이며, 사용자 요청 조건과 한국인의 일반적인 식습관을 고려하여 식단을 구성합니다.

**핵심 규칙:**
1. 출력은 오직 <출력 예시>의 **형태와 패턴**을 **한 글자도 틀리지 않고** 엄격히 준수해야 합니다.
2. 절대 마크다운(**, ## 등)이나 불필요한 구분선을 사용하지 마세요.
3. [음식의 조화]: 각 후보는 **최소 3가지 이상의 음식**을 포함하는 조화로운 조합이어야 합니다.
4. [칼로리]: 아침, 점심, 저녁의 총합 칼로리가 1800kcal ~ 2500kcal 사이에 오도록 스스로 계산을 완료하세요. "조정 필요" 문구는 절대 금지합니다.
5. 절대 간식을 추가하지 마세요. 모든 음식은 <참고 문서>에서 찾은 음식만 사용하세요. 브랜드명은 제거하세요.

<참고 문서> (음식명, 총 칼로리, 영양 정보 등)
{{context}}

<사용자 요청>
{{query}}

<출력 예시> (반드시 이 형식의 텍스트 패턴을 따르세요. 실제 날짜와 칼로리를 계산하여 대체하세요.)
< {today} 식단 추천 >
아침
후보1 : 현미밥 + 닭가슴살 + 시금치나물 (총 550 kcal)
후보2 : 오트밀 + 삶은 계란 + 저지방우유 (총 480 kcal)
후보3 : 곤드레밥 + 소고기미역국 + 깍두기 (총 650 kcal)
점심
후보1 : 잡곡밥 + 제육볶음 + 콩나물국 (총 800 kcal)
후보2 : 치킨텐더샐러드 + 통밀빵 + 요거트 (총 750 kcal)
후보3 : 김치찌개 + 두부부침 + 흑미밥 (총 780 kcal)
저녁
후보1 : 연어스테이크 + 구운채소 + 렌틸콩수프 (총 600 kcal)
후보2 : 현미밥 + 멸치볶음 + 미역국 + 달걀후라이 (총 630 kcal)
후보3 : 닭가슴살 볶음밥 + 단백질쉐이크 (총 650 kcal)
이유 : 근육 증진을 위해 고단백과 건강한 지방 위주로 구성하였으며, 나트륨을 낮게 유지하여 건강을 고려했습니다. 하루 총 칼로리는 [계산된 총합 칼로리]kcal 입니다.
"""

In [ ]:
# -----------------------------
# 3️⃣ Pinecone 검색 후 Context 구성 (LLM이 계산 및 조합하도록)
# -----------------------------
# 쿼리 예시
query = "근육증진형, 고단백, 고지방, 저나트륨 식단 중심으로, 아침/점심/저녁 세 끼만 추천해줘. 각 끼니별로 3가지의 완벽한 식사 조합을 만들고, 하루 총 칼로리가 1800~2500kcal 사이에 오도록 정확히 계산해줘."

query_emb = client.embeddings.create(
    model="text-embedding-3-small",
    input=query
).data[0].embedding

results = index.query(
    vector=query_emb,
    top_k=50,
    namespace="foods-ns1",
    include_metadata=True
)

# Context 구성 수정: 'food_weight' 키 제거
context = "\n".join([
    f"음식명: {m['metadata']['food_name']}, " 
    f"칼로리(총): {m['metadata']['energy_kcal']}kcal, "
    f"단백질: {m['metadata']['protein_g']}g, "
    f"지방: {m['metadata']['fat_g']}g, "
    f"탄수화물: {m['metadata']['carb_g']}g, " 
    f"태그: {m['metadata']['display_tags']}"
    for m in results["matches"]
])

# -----------------------------
# 4️⃣ LLM 실행 코드는 그대로 유지
# -----------------------------
chain = LLMChain(llm=llm, prompt=template)
response = chain.run({"context": context, "query": query})

print(" 추천 결과:")
print(response)

# 수정된 프롬프트

In [14]:
import datetime
today = datetime.date.today().strftime("%Y-%m-%d")

# 4) 프롬프트 템플릿
prompt_template_str = """
당신은 최고 수준의 영양사이며, 사용자 요청 조건과 한국인의 일반적인 식습관을 고려하여 식단을 구성합니다.

핵심 규칙:
1. 출력은 오직 <출력 예시>의 형태와 패턴을 한 글자도 틀리지 않고 엄격히 준수해야 합니다.
2. 절대 마크다운(**, ## 등)이나 불필요한 구분선을 사용하지 마세요.
3. [음식의 조화]: 각 후보는 최소 3가지 이상의 음식을 포함하는 조화로운 조합이어야 합니다.
4. [칼로리]: 아침, 점심, 저녁의 총합 칼로리가 1800kcal ~ 2500kcal 사이에 오도록 스스로 계산을 완료하세요. "조정 필요" 문구는 절대 금지합니다.
5. 모든 음식은 <참고 문서>에서 찾은 음식만 사용하세요. 브랜드명은 제거하세요.

<참고 문서> (음식명, 총 칼로리, 영양 정보 등)
{context}

<사용자 요청>
{query}

<출력 예시>
< {today} 식단 추천 >
아침
후보1 : 현미밥 + 닭가슴살 + 시금치나물 (총 550 kcal)
후보2 : 오트밀 + 삶은 계란 + 저지방우유 (총 480 kcal)
후보3 : 곤드레밥 + 소고기미역국 + 깍두기 (총 650 kcal)
점심
후보1 : 잡곡밥 + 제육볶음 + 콩나물국 (총 800 kcal)
후보2 : 치킨텐더샐러드 + 통밀빵 + 요거트 (총 750 kcal)
후보3 : 김치찌개 + 두부부침 + 흑미밥 (총 780 kcal)
저녁
후보1 : 연어스테이크 + 구운채소 + 렌틸콩수프 (총 600 kcal)
후보2 : 현미밥 + 멸치볶음 + 미역국 + 달걀후라이 (총 630 kcal)
후보3 : 닭가슴살 볶음밥 + 단백질쉐이크 (총 650 kcal)
이유 : 근육 증진을 위해 고단백과 건강한 지방 위주로 구성하였으며, 나트륨을 낮게 유지하여 건강을 고려했습니다. 하루 총 칼로리는 [계산된 총합 칼로리]kcal 입니다.
""".strip()

template = PromptTemplate(
    template=prompt_template_str,
    input_variables=["context", "query", "today"]
)

In [15]:
# 5) Pinecone 검색 → context 구성
user_query = "근육증진형, 고단백, 고지방, 저나트륨 식단 중심으로, 아침/점심/저녁 세 끼만 추천해줘. 각 끼니별로 3가지의 완벽한 식사 조합을 만들고, 하루 총 칼로리가 1800~2500kcal 사이에 오도록 정확히 계산해줘."

# 임베딩 (OpenAI 클라이언트는 oai 사용)
qv = oai.embeddings.create(
    model="text-embedding-3-small",
    input=user_query
).data[0].embedding

# Pinecone 쿼리 (결과 표준화)
res = index.query(
    vector=qv,
    top_k=50,
    namespace=NAMESPACE,
    include_metadata=True
)
res = res.to_dict() if hasattr(res, "to_dict") else res
matches = res.get("matches", [])

# context 문자열 만들기 (결측 방어)
lines = []
for m in matches:
    md = m.get("metadata", {}) or {}
    name = str(md.get("food_name", "")).strip()
    kcal = md.get("energy_kcal", "")
    pro  = md.get("protein_g", "")
    fat  = md.get("fat_g", "")
    carb = md.get("carb_g", "")
    disp = md.get("display_tags", "")
    if name:  # 이름이 있는 것만
        lines.append(
            f"음식명: {name}, 칼로리(총): {kcal}kcal, 단백질: {pro}g, 지방: {fat}g, 탄수화물: {carb}g, 태그: {disp}"
        )
context = "\n".join(lines)

# 6) LLM 실행
chain = LLMChain(llm=llm, prompt=template)
response = chain.run({"context": context, "query": user_query, "today": today})

print("추천 결과:")
print(response)

C:\Users\Admin\AppData\Local\Temp\ipykernel_23804\2397249659.py:37: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=template)
C:\Users\Admin\AppData\Local\Temp\ipykernel_23804\2397249659.py:38: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"context": context, "query": user_query, "today": today})


추천 결과:
< 2025-10-16 식단 추천 >
아침  
후보1 : 잡곡밥 + 오징어채볶음 + 달걀라면 (총 570 kcal)  
후보2 : 검정콩밥 + 땅콩조림 + 열무된장나물 (총 570 kcal)  
후보3 : 현미밥 + 멸치볶음 + 오리백숙 (총 640 kcal)  
점심  
후보1 : 떡볶이 + 잡채 + 조미김 (총 800 kcal)  
후보2 : 고등어석쇠구이 + 잡곡밥 + 미역줄기볶음 (총 780 kcal)  
후보3 : 오징어조림 + 검정콩밥 + 부추전 (총 650 kcal)  
저녁  
후보1 : 간장양념닭다리구이 + 현미밥 + 당근볶음 (총 600 kcal)  
후보2 : 오징어채조림 + 잡곡밥 + 열무나물 (총 700 kcal)  
후보3 : 오징어덮밥 + 멸치볶음 + 묵은지삼겹살볶음 (총 640 kcal)  
이유 : 근육 증진을 위해 고단백과 건강한 지방 위주로 구성하였으며, 나트륨을 낮게 유지하여 건강을 고려했습니다. 하루 총 칼로리는 2300kcal 입니다.
